# 微调 Open AI 模型

本笔记本基于 Open AI 官方 [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) 文档中的最新指导。

微调通过用与特定应用场景相关的额外数据和上下文对基础模型进行再训练，从而提升模型在你应用中的表现。需要注意的是，像 _few shot learning_（少样本学习）和 _retrieval augmented generation_（检索增强生成）这样的提示工程技术，可以通过在默认提示中加入相关数据来提升输出质量。但这些方法受限于目标基础模型的最大 token 窗口大小。

通过微调，我们实际上是用所需的数据对模型本身进行再训练（这样我们可以使用远超最大 token 窗口的样本量），并部署一个_定制_版本的模型，这样在推理时就不再需要提供示例。这不仅提升了我们在提示设计上的灵活性（token 窗口可以用于更多其他内容），还可能降低我们的成本（因为推理时需要发送给模型的 token 数量减少了）。

微调分为四个步骤：
1. 准备训练数据并上传。
1. 运行训练任务，获得微调后的模型。
1. 评估微调模型并反复优化以提升质量。
1. 满意后，将微调模型部署用于推理。

需要注意的是，并非所有基础模型都支持微调——请查阅 [OpenAI 官方文档](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) 获取最新信息。你也可以对已经微调过的模型再次进行微调。在本教程中，我们将以 `gpt-35-turbo` 作为微调的目标基础模型。 

---


### 步骤 1.1：准备你的数据集

我们来构建一个聊天机器人，帮助你通过回答元素相关问题的打油诗来理解元素周期表。在这个简单的教程中，我们只会创建一个数据集，用几个示例来训练模型，展示数据的预期格式。实际应用中，你需要准备更多的样本。如果有现成的开放数据集（适用于你的领域），也可以用来微调，只需重新格式化即可。

由于我们关注的是 `gpt-35-turbo`，并且只需要单轮回复（聊天补全），可以用[这个推荐格式](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst)来创建示例，符合 OpenAI 聊天补全的要求。如果你需要多轮对话内容，可以用[多轮示例格式](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst)，其中包含一个 `weight` 参数，用来标记哪些消息应该在微调过程中使用。

本教程我们采用更简单的单轮格式。数据采用 [jsonl 格式](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst)，每行一个记录，每个记录都是 JSON 格式的对象。下面的代码片段展示了两个示例记录——完整的样本集（10 个例子）请见 [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl)，我们会用它来进行微调演示。**注意：**每条记录必须写在一行内，不能像常规 JSON 文件那样分行。

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

实际应用中，你需要更大的样本集才能获得更好的效果——这需要在回复质量和微调的时间/成本之间做权衡。我们这里用较小的数据集，是为了能快速完成微调，方便演示流程。更复杂的微调教程可以参考 [OpenAI Cookbook 示例](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst)。


### 步骤 1.2 上传你的数据集

使用 Files API 上传数据，[具体方法见此处](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file)。请注意，要运行以下代码，你需要先完成以下步骤：
 - 安装 `openai` Python 包（请确保使用的版本 >=0.28.0，以获得最新功能）
 - 设置 `OPENAI_API_KEY` 环境变量为你的 OpenAI API 密钥
想了解更多信息，请参阅本课程提供的[设置指南](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst)。

现在，运行代码，从本地 JSONL 文件创建一个可上传的文件。


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### 步骤 2.1：使用 SDK 创建微调任务


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### 步骤 2.2：检查作业状态

你可以通过 `client.fine_tuning.jobs` API 做以下几件事：
- `client.fine_tuning.jobs.list(limit=<n>)` - 列出最近的 n 个微调作业
- `client.fine_tuning.jobs.retrieve(<job_id>)` - 获取某个特定微调作业的详细信息
- `client.fine_tuning.jobs.cancel(<job_id>)` - 取消一个微调作业
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - 列出该作业的最多 n 条事件
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

流程的第一步是_验证训练文件_，以确保数据格式正确。


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### 步骤 2.3：跟踪事件以监控进度


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### 步骤2.4：在OpenAI仪表板中查看状态


你也可以通过访问 OpenAI 网站并浏览平台的 _Fine-tuning_ 部分来查看状态。这会显示当前任务的状态，还可以让你追踪之前任务执行的历史记录。在这张截图中，你可以看到上一次执行失败了，而第二次运行成功了。这里的情况是，第一次运行时使用了格式错误的 JSON 文件——修正后，第二次运行顺利完成，并且模型也可以正常使用了。

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.zh.png)


你还可以像下面这样，在可视化仪表板中向下滚动查看更多状态消息和指标：

| 消息 | 指标 |
|:---|:---|
| ![消息](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.zh.png) |  ![指标](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.zh.png)|


### 步骤 3.1：获取 ID 并在代码中测试微调后的模型


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### 步骤 3.2：在 Playground 中加载并测试微调后的模型

现在你可以通过两种方式测试微调后的模型。首先，你可以访问 Playground，并在模型下拉菜单中选择你刚刚微调好的模型。另一种方式是在微调面板中使用“Playground”选项（见上方截图），这会打开一个对比视图，将基础模型和微调模型的版本并排显示，方便你快速评估。

只需填写你在训练数据中使用的系统上下文，并输入你的测试问题。你会发现两边都会同步更新相同的上下文和问题。运行对比后，你就能看到两者输出的不同。_请注意，微调后的模型会按照你示例中提供的格式生成回复，而基础模型则只是遵循系统提示_。

你还会注意到，对比视图会显示每个模型的 token 数量，以及推理所用的时间。**这个例子只是为了演示流程，实际并不代表真实的数据集或场景**。你可能会发现两个示例的 token 数量相同（系统上下文和用户提示一致），但微调模型的推理时间更长（因为是自定义模型）。

在真实场景中，你不会用这样简单的例子，而是会用真实数据（比如用于客户服务的产品目录）进行微调，这样回复的质量会更加明显。在这种情况下，如果想让基础模型达到类似的回复质量，就需要更多的自定义提示工程，这会增加 token 的使用量，也可能导致推理时间变长。_想要尝试的话，可以参考 OpenAI Cookbook 里的微调示例开始实践。_



---

**免责声明**：  
本文件由 AI 翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻译。我们力求准确，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文件应被视为权威来源。对于关键信息，建议使用专业人工翻译。因使用本翻译而产生的任何误解或曲解，我们概不负责。
